# Bayesian analysis and MCMC

## Defining priors

In [1]:
from likelihood import JointPriors

# Defining the priors and the type of distribuition
param_configs_w_frb = {
    'h': ((0.669, 0.7504), 'uniform'),
    'Omega_mh2': ((0.09, 0.2), 'uniform'),
    'Omega_bh2': ((0.02186, 0.02284), 'uniform'), 
    #'Omega_bh2': ((0.02235, 0.00049), 'gaussian'),
    'DM_host_0': ((55, 225), 'uniform'),
    'omega_0': ((-2, 0), 'uniform')
}

param_configs_CPL_frb = {
    'h': ((0.669, 0.7504), 'uniform'),
    'Omega_mh2': ((0.05, 0.2), 'uniform'),
    'Omega_bh2': ((0.02186, 0.02284), 'uniform'), 
    #'Omega_bh2': ((0.02235, 0.00049), 'gaussian'),
    'DM_host_0': ((55, 225), 'uniform'),
    'omega_0': ((-2, 0), 'uniform'),
    'omega_a': ((-2, 2), 'uniform')
}

param_configs_w_sne = {
    'h': ((0.669, 0.7504), 'uniform'),
    'Omega_mh2': ((0.05, 0.2), 'uniform'),
    'omega_0': ((-2, 0), 'uniform')
}

param_configs_CPL_sne = {
    'h': ((0.669, 0.7504), 'uniform'),
    'Omega_mh2': ((0.05, 0.2), 'uniform'),
    'omega_0': ((-2, 0), 'uniform'),
    'omega_a': ((-2, 2), 'uniform')
}

P_w_frb = JointPriors(param_configs_w_frb)
P_w_sne = JointPriors(param_configs_w_sne)
P_CPL_frb = JointPriors(param_configs_CPL_frb)
P_CPL_sne = JointPriors(param_configs_CPL_sne)

### Models and parameters

In [2]:
from likelihood import JointLikelihoodFunction
from equations import DM_EXT_model, Modulus_sne, Hubble

model = DM_EXT_model()
mu_wCDM = Modulus_sne()
H_model = Hubble()

# Creating an instance of JointLikelihoodFunction
LF_w_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, h, Omega_mh2, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,  
        Omega_mh2=Omega_mh2,
        h=h,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_w_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, h, Omega_mh2, omega_0: mu_wCDM.Modulo_std(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_w_cc = JointLikelihoodFunction(
    { 'CC': lambda z, h, Omega_mh2, omega_0: H_model.H_func(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_w_sne_frb_cc = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, h, Omega_mh2, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        h=h, 
        Omega_mh2=Omega_mh2,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ),

    'SNe': lambda z, h, Omega_mh2, omega_0: mu_wCDM.Modulo_std(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ),

    'CC': lambda z, h, Omega_mh2, omega_0: H_model.H_func(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    },

    shared_params = ["h", "Omega_mh2", "omega_0"]
)

LF_CPL_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, h, Omega_mh2, DM_host_0, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        h=h, 
        Omega_mh2=Omega_mh2,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, h, Omega_mh2, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_cc = JointLikelihoodFunction(
    { 'CC': lambda z, h, Omega_mh2, omega_0, omega_a: H_model.H_func(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_sne_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, h, Omega_mh2, DM_host_0, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        h=h, 
        Omega_mh2=Omega_mh2,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ),

    'SNe': lambda z, h, Omega_mh2, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ),

    'CC': lambda z, h, Omega_mh2, omega_0, omega_a: H_model.H_func(
        z=z,
        Omega_mh2=Omega_mh2,
        h=h,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    },
    shared_params = ["h", "Omega_mh2", "omega_0", "omega_a"]
)

## Preparing the samples

### Analysis for 66 FRBs

In [3]:
from obs_data import FRB_data, SNe_data, H_data
import ultranest

# Instantiate the FRB_data class for 66 FRBs
frb_data = FRB_data(n_frb=66)

sne_data = SNe_data(sample_sne='Pantheon+SH0ES')

z_sne, mu_sne, cov_matrix = sne_data.load_data()

# Call the select_data method to get the observed data
z_frb, DM_ext, sigma_ext = frb_data.select_data()

Hz_data = H_data()

z_val, H_val, errors = Hz_data.H_z_data()

# Configuring the ultranest samplers
sampler_w_frb = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_frb.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_w_frb.prior_transform
)

sampler_w_sne = ultranest.ReactiveNestedSampler(
    P_w_sne.param_names,
    lambda params: LF_w_sne.log_likelihood(
        dict(zip(P_w_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w_sne.prior_transform
)

sampler_w_cc = ultranest.ReactiveNestedSampler(
    P_w_sne.param_names,
    lambda params: LF_w_cc.log_likelihood(
        dict(zip(P_w_sne.param_names, params)),
        {
            'CC': (z_val, H_val, errors)
        }
    ),
    P_w_sne.prior_transform
)

sampler_w_sne_frb_cc = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_sne_frb_cc.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'SNe': (z_sne, mu_sne, cov_matrix),
            'CC': (z_val, H_val, errors)
        }
    ),
    P_w_frb.prior_transform
)

sampler_CPL_frb = ultranest.ReactiveNestedSampler(
    P_CPL_frb.param_names,
    lambda params: LF_CPL_frb.log_likelihood(
        dict(zip(P_CPL_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_CPL_frb.prior_transform
)

sampler_CPL_sne = ultranest.ReactiveNestedSampler(
    P_CPL_sne.param_names,
    lambda params: LF_CPL_sne.log_likelihood(
        dict(zip(P_CPL_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_CPL_sne.prior_transform
)

sampler_CPL_cc = ultranest.ReactiveNestedSampler(
    P_CPL_sne.param_names,
    lambda params: LF_CPL_cc.log_likelihood(
        dict(zip(P_CPL_sne.param_names, params)),
        {
            'CC': (z_val, H_val, errors)
        }
    ),
    P_CPL_sne.prior_transform
)

sampler_CPL_sne_frb_cc = ultranest.ReactiveNestedSampler(
    P_CPL_frb.param_names,
    lambda params: LF_CPL_sne_frb.log_likelihood(
        dict(zip(P_CPL_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'SNe': (z_sne, mu_sne, cov_matrix),
            'CC': (z_val, H_val, errors)
        }
    ),
    P_CPL_frb.prior_transform
)

In [4]:
wCDM_frb = sampler_w_frb.run(min_num_live_points=400)
sampler_w_frb.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-8e+01   [-83.1837..-83.1831]*| it/evals=3856/12341 eff=32.2921% N=400 
[ultranest] Likelihood function evaluations: 12342
[ultranest]   logZ = -88.15 +- 0.05005
[ultranest] Effective samples strategy satisfied (ESS = 2196.8, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.06 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.05, need <0.5)
[ultranest]   logZ error budget: single: 0.09 bs:0.05 tail:0.01 total:0.05 required:<0.50
[ultranest] done iterating.

logZ = -88.147 +- 0.118
  single instance: logZ = -88.147 +- 0.087
  bootstrapped   : logZ = -88.148 +- 0.118
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.669 │▇▆▅▆▅▅▄▄▄▄▄▄▃▂▃▃▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁│0.750     0.692 +- 0.019
    Omega_mh2           : 0.090 │▇▇▇▆▇▆▇▆▅▅▅▄▅▄▄▄▃▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁│0.200     0.120 +- 0.024
    Omega_bh2           : 0.02186│▂

In [ ]:
wCDM_sne = sampler_w_sne.run(min_num_live_points=400)
sampler_w_sne.print_results()

[ultranest] Sampling 400 live points from prior ...


In [5]:
wCDM_cc = sampler_w_cc.run(min_num_live_points=400)
sampler_w_cc.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-7  26 [-7.2959..-7.2958]*| it/evals=2970/6216 eff=51.0660% N=400   
[ultranest] Likelihood function evaluations: 6221
[ultranest]   logZ = -10.13 +- 0.04184
[ultranest] Effective samples strategy satisfied (ESS = 1522.5, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.09 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.04, need <0.5)
[ultranest]   logZ error budget: single: 0.07 bs:0.04 tail:0.01 total:0.04 required:<0.50
[ultranest] done iterating.

logZ = -10.110 +- 0.069
  single instance: logZ = -10.110 +- 0.068
  bootstrapped   : logZ = -10.128 +- 0.068
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.669 │▆▅▅▆▇▆▅▅▆▇▆▆▆▇▅▅▇▅▆▆▆▇▅▆▅▆▆▄▅▅▆▆▅▃▅▅▅▅▄│0.750     0.708 +- 0.023
    Omega_mh2           : 0.050 │▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▃▄▄▄▄▄▅▄▅▆▇▇▇▅▆▅▄▄▂│0.200     0.158 +- 0.026
    omega_0             : -2.00 │▁▂▂▂▂▃

In [9]:
wCDM_sne_frb_cc = sampler_w_sne_frb_cc.run(min_num_live_points=400)
sampler_w_sne_frb_cc.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-1e+03  052.53 [-1052.9149..-1052.9145]*| it/evals=5918/23063 eff=26.1130% N=400 
[ultranest] Likelihood function evaluations: 23063
[ultranest]   logZ = -1063 +- 0.1046
[ultranest] Effective samples strategy satisfied (ESS = 2390.1, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.07 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.11, need <0.5)
[ultranest]   logZ error budget: single: 0.14 bs:0.10 tail:0.01 total:0.11 required:<0.50
[ultranest] done iterating.

logZ = -1062.986 +- 0.239
  single instance: logZ = -1062.986 +- 0.138
  bootstrapped   : logZ = -1062.984 +- 0.238
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.7143│ ▁▁▁▁▁▁▁▁▁▁▂▂▃▄▅▆▇▇▇▆▇▅▅▅▃▂▂▁▁▁▁▁▁▁▁ ▁ │0.7348    0.7245 +- 0.0024
    Omega_mh2           : 0.090 │▇▇▇▆▆▆▅▅▄▅▄▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁   ▁▁▁│0.200     0.112 +- 0.016
    Omega_bh2      

In [6]:
CPL_frb = sampler_CPL_frb.run(min_num_live_points=400)
sampler_CPL_frb.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-8e+01   [-83.2247..-83.2244]*| it/evals=3652/15758 eff=23.7791% N=400 
[ultranest] Likelihood function evaluations: 15758
[ultranest]   logZ = -87.65 +- 0.05808
[ultranest] Effective samples strategy satisfied (ESS = 2013.5, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.07 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.06, need <0.5)
[ultranest]   logZ error budget: single: 0.08 bs:0.06 tail:0.01 total:0.06 required:<0.50
[ultranest] done iterating.

logZ = -87.654 +- 0.119
  single instance: logZ = -87.654 +- 0.080
  bootstrapped   : logZ = -87.653 +- 0.118
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.669 │▇▇▆▅▇▆▅▄▅▄▅▄▃▄▃▃▂▄▃▂▃▂▂▃▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁│0.750     0.694 +- 0.020
    Omega_mh2           : 0.050 │▅▆▅▇▇▇▇▇▆▅▆▇▅▅▅▅▅▄▄▄▃▃▃▃▂▂▃▂▁▁▂▁▁▁▁▁▁▁▁│0.200     0.097 +- 0.032
    Omega_bh2           : 0.02186│▃

In [10]:
CPL_sne = sampler_CPL_sne.run(min_num_live_points=400)
sampler_CPL_sne.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-9e+02  .22 [-946.3980..-946.3979]*| it/evals=5119/17697 eff=29.5947% N=400  
[ultranest] Likelihood function evaluations: 17697
[ultranest]   logZ = -954.5 +- 0.067
[ultranest] Effective samples strategy satisfied (ESS = 1938.9, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.06 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.07, need <0.5)
[ultranest]   logZ error budget: single: 0.13 bs:0.07 tail:0.01 total:0.07 required:<0.50
[ultranest] done iterating.

logZ = -954.535 +- 0.113
  single instance: logZ = -954.535 +- 0.127
  bootstrapped   : logZ = -954.540 +- 0.113
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.7087│ ▁ ▁▁▁▁▁▁▂▂▃▄▄▅▇▆▇▇▇▇▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁ │0.7329    0.7203 +- 0.0030
    Omega_mh2           : 0.050 │▁▁▁▁▂▂▁▂▂▂▁▁▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▇▆▇▅▇▆▅▅▆▅▄│0.200     0.147 +- 0.035
    omega_0             : 

In [7]:
CPL_cc = sampler_CPL_cc.run(min_num_live_points=400)
sampler_CPL_cc.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-7  21 [-7.2613..-7.2613]*| it/evals=2930/19541 eff=15.3075% N=400   
[ultranest] Likelihood function evaluations: 19705
[ultranest]   logZ = -9.974 +- 0.05167
[ultranest] Effective samples strategy satisfied (ESS = 1481.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.07 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.05, need <0.5)
[ultranest]   logZ error budget: single: 0.07 bs:0.05 tail:0.01 total:0.05 required:<0.50
[ultranest] done iterating.

logZ = -9.983 +- 0.182
  single instance: logZ = -9.983 +- 0.068
  bootstrapped   : logZ = -9.974 +- 0.182
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.669 │▄▅▇▅▄▅▄▅▅▅▆▇▆▅▆▅▇▆▇▇▅▆▇▆▆▇▅▅▅▅▅▆▆▄▅▇▅▅▅│0.750     0.710 +- 0.023
    Omega_mh2           : 0.050 │▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▃▃▃▄▄▅▆▇▇▆▇▆▇▅▅▅▄▃│0.200     0.154 +- 0.032
    omega_0             : -2.00 │▁▁▂▂▃▃▃

In [11]:
CPL_sne_frb_cc = sampler_CPL_sne_frb_cc.run(min_num_live_points=400)
sampler_CPL_sne_frb_cc.print_results()

[ultranest] Sampling 400 live points from prior ...


[ultranest] Explored until L=-1e+03  050.89 [-1051.3997..-1051.3996]*| it/evals=6160/41517 eff=14.9816% N=400 
[ultranest] Likelihood function evaluations: 41517
[ultranest]   logZ = -1062 +- 0.1184
[ultranest] Effective samples strategy satisfied (ESS = 2420.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.05 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.12, need <0.5)
[ultranest]   logZ error budget: single: 0.14 bs:0.12 tail:0.01 total:0.12 required:<0.50
[ultranest] done iterating.

logZ = -1062.065 +- 0.266
  single instance: logZ = -1062.065 +- 0.142
  bootstrapped   : logZ = -1062.112 +- 0.266
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    h                   : 0.7109│ ▁▁▁▁▁▁▁▂▂▃▄▄▅▇▇▇▇▇▇▇▅▅▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁ │0.7332    0.7211 +- 0.0029
    Omega_mh2           : 0.050 │▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▄▄▄▅▅▆▆▇▇▇▆▅▄▄▃▂▂▁▁▁│0.200     0.144 +- 0.027
    Omega_bh2      

In [30]:
from getdist import plots, MCSamples

# Extraindo amostras dos resultados para frb
samples_w_frb = wCDM_frb['samples']
samples_CPL_frb = CPL_frb['samples']

labels_w_frb = ['h', '\\Omega_{b}h^2', '\\Omega_{m}h^2', 'DM_{host,0}', '\\omega_0']
names_w_frb = P_w_frb.param_names
mcsamples_w_frb = MCSamples(samples=samples_w_frb, names=names_w_frb, labels=labels_w_frb)

labels_CPL_frb = ['h', '\\Omega_{b}h^2', '\\Omega_{m}h^2', 'DM_{host,0}', '\\omega_0', '\\omega_a']
names_CPL_frb = P_CPL_frb.param_names
mcsamples_CPL_frb = MCSamples(samples=samples_CPL_frb, names=names_CPL_frb, labels=labels_CPL_frb)

# Extraindo amostras dos resultados para sne
samples_w_sne = wCDM_sne['samples']
samples_CPL_sne = CPL_sne['samples']

labels_w_sne = ['h', '\\Omega_{m}h^2', '\\omega_0']
names_w_sne = P_w_sne.param_names
mcsamples_w_sne = MCSamples(samples=samples_w_sne, names=names_w_sne, labels=labels_w_sne)

labels_CPL_sne = ['h', '\\Omega_{m}h^2', '\\omega_0', '\\omega_a']
names_CPL_sne = P_CPL_sne.param_names
mcsamples_CPL_sne = MCSamples(samples=samples_CPL_sne, names=names_CPL_sne, labels=labels_CPL_sne)

# Extraindo amostras dos resultados para cc
samples_w_cc = wCDM_cc['samples']
samples_CPL_cc = CPL_cc['samples']

mcsamples_w_cc = MCSamples(samples=samples_w_cc, names=names_w_sne, labels=labels_w_sne)

mcsamples_CPL_cc = MCSamples(samples=samples_CPL_cc, names=names_CPL_sne, labels=labels_CPL_sne)

# Extraindo amostras dos resultados para sne+frb+cc

samples_w_sne_frb_cc = wCDM_sne_frb_cc['samples']
samples_CPL_sne_frb_cc = CPL_sne_frb_cc['samples']

mcsamples_w_sne_frb_cc = MCSamples(samples=samples_w_sne_frb_cc, names=names_w_frb, labels=labels_w_frb)
mcsamples_CPL_sne_frb_cc = MCSamples(samples=samples_CPL_sne_frb_cc, names=names_CPL_frb, labels=labels_CPL_frb)



Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in


In [54]:
mcsamples_w_frb.getMargeStats().saveAsText('Results_mcmc/marginalized_params_w_frb.txt')
mcsamples_w_cc.getMargeStats().saveAsText('Results_mcmc/marginalized_params_w_cc.txt')
mcsamples_w_sne.getMargeStats().saveAsText('Results_mcmc/marginalized_params_w_sne.txt')
mcsamples_w_sne_frb_cc.getMargeStats().saveAsText('Results_mcmc/marginalized_params_w_frb_sne_cc.txt')
mcsamples_CPL_frb.getMargeStats().saveAsText('Results_mcmc/marginalized_params_CPL_frb.txt')
mcsamples_CPL_cc.getMargeStats().saveAsText('Results_mcmc/marginalized_params_CPL_cc.txt')
mcsamples_CPL_sne.getMargeStats().saveAsText('Results_mcmc/marginalized_params_CPL_sne.txt')
mcsamples_CPL_sne_frb_cc.getMargeStats().saveAsText('Results_mcmc/marginalized_params_CPL_frb_sne_cc.txt')

In [53]:
import pandas as pd

# Dicionário de amostras e nomes de arquivos
amostras = {
    "w_frb": samples_w_frb,
    "CPL_frb": samples_CPL_frb,
    "w_sne": samples_w_sne,
    "CPL_sne": samples_CPL_sne,
    "w_cc": samples_w_cc,
    "CPL_cc": samples_CPL_cc,
    "w_sne_frb_cc": samples_w_sne_frb_cc,
    "CPL_sne_frb_cc": samples_CPL_sne_frb_cc,
}

# Salva cada conjunto de amostras em arquivos CSV
for nome, dados in amostras.items():
    df = pd.DataFrame(dados)
    df.to_csv(f"Samplers/{nome}.csv", index=False)


In [ ]:
"""import pandas as pd
df1_frb = pd.read_csv("Samplers/samples1_frb.csv")
df1_sne = pd.read_csv("Samplers/samples1_sne.csv")
df1_sne_frb = pd.read_csv("Samplers/samples1_sne_frb.csv")
df2_frb = pd.read_csv("Samplers/samples2_frb.csv")
df2_sne = pd.read_csv("Samplers/samples2_sne.csv")
df2_sne_frb = pd.read_csv("Samplers/samples2_sne_frb.csv")"""

In [ ]:
"""from getdist import plots, MCSamples

# Extraindo amostras dos resultados para frb
samples1_frb = df1_frb.values
samples2_frb = df2_frb.values

labels1 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0']
names1 = P_w_frb.param_names
mcsamples1_frb = MCSamples(samples=samples1_frb, names=names1, labels=labels1)

labels2 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0', '\\omega_a']
names2 = P_CPL_frb.param_names
mcsamples2_frb = MCSamples(samples=samples2_frb, names=names2, labels=labels2)

# Extraindo amostras dos resultados para sne
samples1_sne = df1_sne.values
samples2_sne = df2_sne.values

labels3 = ['H_0', '\\Omega_m', '\\omega_0']
names3 = P_w_sne.param_names
mcsamples1_sne = MCSamples(samples=samples1_sne, names=names3, labels=labels3)

labels4 = ['H_0', '\\Omega_m', '\\omega_0', '\\omega_a']
names4 = P_CPL_sne.param_names
mcsamples2_sne = MCSamples(samples=samples2_sne, names=names4, labels=labels4)

# Extraindo amostras dos resultados para sne+frb

samples1_sne_frb = df1_sne_frb.values
samples2_sne_frb = df2_sne_frb.values

mcsamples1_sne_frb = MCSamples(samples=samples1_sne_frb, names=names1, labels=labels1)
mcsamples2_sne_frb = MCSamples(samples=samples2_sne_frb, names=names2, labels=labels2)

"""

In [44]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples_w_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_w_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_w_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_w_sne_frb_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples_w_frb, mcsamples_w_sne, mcsamples_w_cc, mcsamples_w_sne_frb_cc], ['h', 'omega_0'], filled=True, contour_colors=['green', 'red', 'blue', 'purple'], 
                legend_labels=['FRB', 'SNe', 'CC', 'FRB + SNe + CC'])
g.export('Figuras/wCDM_joint.png', dpi=600)

In [45]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples_CPL_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_CPL_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_CPL_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_CPL_sne_frb_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples_CPL_frb, mcsamples_CPL_sne, mcsamples_CPL_cc, mcsamples_CPL_sne_frb_cc], ['h', 'omega_0', 'omega_a'], filled=True, contour_colors=['green', 'red', 'blue', 'purple'], 
                legend_labels=['FRB', 'SNe', 'CC', 'FRB + SNe + CC'])
g.export('Figuras/w0waCDM_joint.png', dpi=600)

In [47]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
# mcsamples_w_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
# mcsamples_w_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
# mcsamples_w_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_w_sne_frb_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples_w_sne_frb_cc], filled=True, contour_colors=['purple'], 
                legend_labels=['FRB + SNe + CC'])
g.export('Figuras/wCDM_only_joint.png', dpi=600)

## Model Comparison

In [48]:
samples = {
    "Sample_frb": {
        "results": {
            'wCDM': {'logz': wCDM_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_frb['logz'], 'num_params': 6},
        }
    },
    "Sample_sne": {
        "results": {
            'wCDM': {'logz': wCDM_sne['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne['logz'], 'num_params': 6},
        }
    },
    "Sample_cc": {
        "results": {
            'wCDM': {'logz': wCDM_cc['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_cc['logz'], 'num_params': 6},
        }
    },
    "Sample_sne_frb_cc": {
        "results": {
            'wCDM': {'logz': wCDM_sne_frb_cc['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne_frb_cc['logz'], 'num_params': 6},
        }
    }
}

In [49]:
from bayesian_analysis import ModelComparison

comparison = ModelComparison(samples)
comparison.run_comparisons(save_to_file='Results_mcmc/comparisons_output_joint.txt')

In [51]:
from bayesian_analysis import SaveResults

# Initialize the class with the desired output file
results_saver = SaveResults("Results_mcmc/w0wa_results_frb_sne_cc.txt")

# Optional: Clear the file if you want to start fresh
results_saver.reset_file()

results_saver.save_to_txt(mcsamples_w_frb, 'wCDM_frb')
results_saver.save_to_txt(mcsamples_CPL_frb, 'CPL_frb')
results_saver.save_to_txt(mcsamples_w_sne, 'wCDM_sne')
results_saver.save_to_txt(mcsamples_CPL_sne, 'CPL_sne')
results_saver.save_to_txt(mcsamples_w_cc, 'wCDM_cc')
results_saver.save_to_txt(mcsamples_CPL_cc, 'CPL_cc')
results_saver.save_to_txt(mcsamples_w_sne_frb_cc, 'wCDM_joint')
results_saver.save_to_txt(mcsamples_CPL_sne_frb_cc, 'CPL_joint')


In [52]:
from bayesian_analysis import CompareMCMCResults

comparator = CompareMCMCResults('Results_mcmc/mcmc_comparison_results_w0wa.txt')

comparator.reset_file()

# Faz a comparação entre os dois modelos
comparator.compare_errors(mcsamples_w_frb, mcsamples_w_sne_frb_cc, "FRB", "w-joint")
comparator.compare_errors(mcsamples_w_cc, mcsamples_w_sne_frb_cc, "CC", "w-joint")
comparator.compare_errors(mcsamples_w_sne, mcsamples_w_sne_frb_cc, "SNe", "w-joint")
comparator.compare_errors(mcsamples_CPL_frb, mcsamples_CPL_sne_frb_cc, "FRB", "CPL-joint")
comparator.compare_errors(mcsamples_CPL_cc, mcsamples_CPL_sne_frb_cc, "CC", "CPL-joint")
comparator.compare_errors(mcsamples_CPL_sne, mcsamples_CPL_sne_frb_cc, "SNe", "CPL-joint")

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_16, y=dm_ext_obs_16, fmt='o', alpha=0.6, color='red', label='16 FRBs', ms=2)

z_test = np.linspace(0, 1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0, omega_a  in sampler_p2_16.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'))

band.line(color='k', linestyle='-', label='CPL parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_16.png', format='png', dpi=600)"""

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_63, y=dm_ext_obs_63, fmt='o', alpha=0.6, color='red', label='63 FRBs', ms=2)

z_test = np.linspace(0, 1.1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0  in sampler_constant_63.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant')
    )

band.line(color='k', linestyle='-', label='Constant parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_63.png', format='png', dpi=600)"""